In [51]:
import pandas as pd
import geopandas as gpd

In [52]:
# Panda settings for showing data (this is foremost done to more easily explore the data while processing it)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [53]:
input_folder = "E://bomen_data//input//"
output_folder = "E://bomen_data//output//"

Translating the inspection data into English

In [54]:
tree_inspections = pd.read_csv(input_folder+'bomen-veiligheidsinspecties-2024-11.csv')

In [55]:
tree_inspections['Advice'] = tree_inspections['Snoeiwijze'].replace({
    'Begeleidingssnoei': 'Formative pruning',
    'Onderhoudssnoei': 'Maintenance pruning', # height 18m + & Trunk diameter 0,5 meter + IS high risk 
    'Knotten': 'Coppicing' ,
    'Fruitsnoei': 'Fruit pruning',
    'Terugzetten (extensief)': 'Selective thinning',
    'Niet van toepassing': 'Not applicable',
    'Terugzetten (intensief)': 'Systematic thinning',
    'Scheren | knippen': 'Trimming | cutting',
    'Kandelaberen': 'Pollarding', # high risk
    'Innemen kroon' : 'Reduding crown' # high risk
})

In [56]:
tree_inspections.rename(columns={'Boomid': 'TreeId'}, inplace=True)
tree_inspections.rename(columns={'Inspecterendeorganisatie': 'Inspecting_company'}, inplace=True)
tree_inspections.rename(columns={'Inspectiedatum': 'Inspection_date'}, inplace=True)

In [57]:
selected_columns = ['Id', 'TreeId','Inspecting_company','Inspection_date','Advice']
tree_inspections.to_csv(output_folder+'tree_inspections_11_2024.csv', columns=selected_columns, index=False)

Translating the tree data of Amsterdam from november 2024 data into English: source -> https://api.data.amsterdam.nl/v1/bomen/stamgegevens/?_format=geojson (nov 2024)

In [ ]:
tree_data_memory_nov24 = gpd.read_file(input_folder+'bomen-stamgegevens_2024_11.json')


In [ ]:
tree_data_nov24 = tree_data_memory_nov24 

In [ ]:
tree_data_nov24.head(5)

In [ ]:
diameter_trunks = tree_data_nov24['stamdiameterklasse'].unique()
diameter_trunks

In [ ]:
tree_data_nov24['diameter_tree_trunk'] = tree_data_nov24['stamdiameterklasse'].replace({
     '0,2 tot 0,3 m.': "0.2 to 0.3 m.",
     '0,1 tot 0,2 m.': "0.1 to 0.2 m.",
     '0,3 tot 0,5 m.': "0.3 to 0.5 m.",
     '0,5 tot 1 m.': "0.5 to 1 m.",
     '1,0 tot 1,5 m.':"1.0 to 1.5 m.",
     'Onbekend': "unknown",
     '1,5 m. en groter': "1.5 m. and bigger",
     'tot 0,1 m.': "smaller than 0.1 m."
})

In [ ]:
tree_heigth = tree_data_nov24['boomhoogteklasseActueel'].unique()
tree_heigth

In [ ]:
tree_data_nov24['height_of_tree'] = tree_data_nov24['boomhoogteklasseActueel'].replace({
     'e. 15 tot 18 m.': 'e. 15 to 18 ',
     'c. 9 tot 12 m.': 'c. 9 to 12 m.' ,
     'b. 6 tot 9 m.': 'b. 6 to 9 m.' ,
     'd. 12 tot 15 m.': 'd. 12 to 15 m.',
     'a. tot 6 m.': 'a. to 6 m.' ,
     'f. 18 tot 24 m.': 'f. 18 to 24 m.',
     'q. Niet van toepassing': 'q. Not applicable',
     'g. 24 m. en hoger': 'g. 24 m. and higher',
     'p. Niet te beoordelen': 'p. Impossible to determine'  ,
     'r. Niet van toepassing (te vellen boom)': 'r. Not applicabel (tree need to be removed)' ,
     'ZZ_Vervallen 6 tot 12 m.': 'ZZ_Discard 6 to 12 m.' ,
     'ZZ_Vervallen 12 tot 18 m.': 'ZZ_Discard 12 to 18 m.'
})

In [ ]:
tree_heigth = tree_data_nov24['typeObject'].unique()
tree_heigth

In [ ]:
tree_data_nov24['object_type'] = tree_data_nov24['typeObject'].replace({
     'Boom niet vrij uitgroeiend': 'non free growing tree',
     'Boom vrij uitgroeiend':'free growing tree',
     'Stobbe':'Stump/Stool',
     'Gekandelaberde boom': 'Pollarded tree',
     'Knotboom': 'Coppice tree',
     'Vormboom': 'Topiary tree',
     'Fruitboom': 'Fruit tree',
     'Leiboom':'Espalier tree',
     'Bosplantsoen': 'bare rooted woodland mix',
     'Aanwijzingsteken' : 'Indication sign'

})

In [ ]:
tree_data_nov24.head(5)

In [ ]:
tree_data_nov24.rename(columns={'typeEigenaarPlus': 'maintainer'}, inplace=True) #For educational purposes we parsed this to a new field called maintainer instead of owner
tree_data_nov24.rename(columns={'jaarVanAanleg': 'year_planted'}, inplace=True)
tree_data_nov24.rename(columns={'soortnaam': 'botanical_name'}, inplace=True)
tree_data_nov24.rename(columns={'soortnaamTop': 'name'}, inplace=True)
tree_data_nov24.rename(columns={'soortnaam': 'botanical_name'}, inplace=True)

tree_data_nov24 = tree_data_nov24.drop(columns=['gbdBuurtId','typeObject','boomhoogteklasseActueel','stamdiameterklasse','typeBeheerderPlus','standplaats', 'standplaatsGedetailleerd', 'typeSoortnaam','soortnaamKort'])

In [ ]:
tree_data_nov24.to_file(output_folder+'trees_nov_2024.geojson', driver='GeoJSON')

Translating the tree data from October 2024 data into English obtained from https://maps.amsterdam.nl/open_geodata/ (currently, nov 2024 not available anymore)

In [ ]:
tree_data_memory_oct24 = gpd.read_file(input_folder+'Trees_amsterdam_2024_10.gpkg')


In [ ]:
tree_data_oct24 =tree_data_memory_oct24

In [ ]:
tree_data_oct24.head()

In [ ]:
tree_data_oct24['current_height_of_tree'] = tree_data_oct24['Boomhoogte'].replace({
     '15 tot 18 m.': '15 to 18 ',
     '9 tot 12 m.': '9 to 12 m.' ,
     '6 tot 9 m.': '6 to 9 m.' ,
     '12 tot 15 m.': '12 to 15 m.',
     'tot 6 m.': 'to 6 m.' ,
     '18 tot 24 m.': '18 to 24 m.',
     'Niet van toepassing': 'Not applicable',
     '24 m. en hoger': '24 m. and higher',
     'Niet te beoordelen': 'Impossible to determine'  ,
     'Niet van toepassing (te vellen': 'Not applicabel (tree need to be removed)' ,
     'Vervallen 6 tot 12 m.': 'Discard 6 to 12 m.' ,
     'Vervallen 12 tot 18 m.': 'Discard 12 to 18 m.'
})

In [ ]:
# https://www.vdberk.com/ has been used as a source to translate Dutch into English

tree_data_oct24['type_detail'] = tree_data_oct24['Type_detail'].replace({
    'Boom niet vrij uitgroeiend':'non free growing tree', 
    'Boom vrij uitgroeiend':'free growing tree',
    'Knotboom':'Coppice tree',
    'Vormboom - Blokboom':'Topiary tree - block', 
    'Gekandelaberde boom': 'Pollarded tree',
    'Fruitboom - Laagstam':'Fruit tree - halfstem', 
    'Vormboom - Bolboom':'Topiary tree - sphere',
    'Vormboom - Geschoren boom':'Topiary tree - trimmed', 
    'Vormboom': 'Topiary tree',
    'Fruitboom': 'Fruit tree',
    'Vormboom - Dakboom':'Topiary tree - roof', 
    'Fruitboom - Hoogstam':'Fruit tree - clearstem', 
    'Leiboom - Geknipte leiboom':'Espalier tree',
    'Boom niet vrij uitgroeiend - Bolboo': 'non free growing tree', 
    'Leiboom':'Espalier tree',
    'Bosplantsoen - Boomvormers':'Bare rooted woodland mix'
})

In [ ]:
tree_data_oct24.head()

In [ ]:
tree_data_oct24['Beheerder'] = tree_data_oct24['Beheerder'].replace({
    'Gemeente':'Gemeente Amsterdam'}) 

In [ ]:
tree_data_oct24.rename(columns={'Boomnummer': 'tree_number'}, inplace=True)
tree_data_oct24.rename(columns={'Soortnaam': 'botanical_name'}, inplace=True)
tree_data_oct24.rename(columns={'SoortnaamKortNL': 'Dutch_name'}, inplace=True)
tree_data_oct24.rename(columns={'Jaar_van_aanleg': 'year_planted'}, inplace=True)
tree_data_oct24.rename(columns={'Beheerder': 'maintainer'}, inplace=True)

tree_data_oct24 = tree_data_oct24.drop(columns=['SoortnaamTop15','Type_detail','Standplaats_detail', 'Boomhoogte'])

In [ ]:
tree_data_oct24.to_file(output_folder+'tree_data_oct24.gpkg', layer='oct_2024', driver="GPKG")

Translating the tree data from October 2023 data into English obtained from https://maps.amsterdam.nl/open_geodata/ (currently, nov 2024 not available anymore)

In [ ]:
tree_data_memory_sept23 = gpd.read_file(input_folder+'Trees_Amsterdam_2023_09.shp')

In [ ]:
tree_data_sept23 = tree_data_memory_sept23

In [ ]:
tree_data_sept23.head()

In [ ]:
unqiue = tree_data_sept23['typeObject'].unique()
unqiue

In [ ]:
tree_data_sept23['height_of_tree'] = tree_data_sept23['boomhoogte'].replace({
    'b. 6 tot 9 m.': 'b. 6 to 9 m.' ,
    'c. 9 tot 12 m.': 'c. 9 to 12 m.' ,
    'a. tot 6 m.': 'a. to 6 m.' ,
    'q. Niet van toepassing': 'q. Not applicable', 
    'e. 15 tot 18 m.': 'e. 15 to 18 ',
    'd. 12 tot 15 m.': 'd. 12 to 15 m.',
    'f. 18 tot 24 m.': 'f. 18 to 24 m.',
    'g. 24 m. en hoger': 'g. 24 m. and higher'
})

In [ ]:
tree_data_sept23['diameter_tree_trunk'] = tree_data_sept23['stamdiamet'].replace({
     '0,2 tot 0,3 m.': "0.2 to 0.3 m.",
     '0,1 tot 0,2 m.': "0.1 to 0.2 m.",
     '0,3 tot 0,5 m.': "0.3 to 0.5 m.",
     '0,5 tot 1 m.': "0.5 to 1 m.",
     '1,0 tot 1,5 m.':"1.0 to 1.5 m.",
     'Onbekend': "unknown",
     '1,5 m. en groter': "1.5 m. and bigger",
     'tot 0,1 m.': "smaller than 0.1 m."
})

In [ ]:
tree_data_sept23['object_type'] = tree_data_sept23['typeObject'].replace({
     'Boom niet vrij uitgroeiend': 'non free growing tree',
     'Boom vrij uitgroeiend':'free growing tree',
     'Stobbe':'Stump/Stool',
     'Gekandelaberde boom': 'Pollarded tree',
     'Knotboom': 'Coppice tree',
     'Vormboom': 'Topiary tree',
     'Fruitboom': 'Fruit tree',
     'Leiboom':'Espalier tree',
     'Bosplantsoen': 'Bare rooted woodland mix',
     'Aanwijzingsteken' : 'Indication sign'

})

In [ ]:
tree_data_sept23.rename(columns={'soortnaam': 'botanical_name'}, inplace=True)
tree_data_sept23.rename(columns={'SoortnaamN': 'Dutch_name'}, inplace=True)
tree_data_sept23.rename(columns={'jaarVanAan': 'year_planted'}, inplace=True)


In [ ]:

tree_data_sept23 = tree_data_sept23.drop(columns=['soortnaamK','boomhoogte','stamdiamet', 'typeObject'])

In [ ]:
tree_data_sept23.to_file(output_folder+"tree_data_sept23", driver="ESRI Shapefile")